 Todo:
 1. check the order of the preprocess. what to do first === DONE ===
 2. create pipeline 
 3. hyper param search
 4. after findind hyper-params, add fit on all data (include val) 
 5. remove noisy samples: one option - remove rows with num of nan > trash
 6. feature selection (annova, pca) 
 7. check for cat boost 
 8. check SMOTETomek and undersample & SMOTENC

In [101]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from copy import copy
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

In [102]:
def read_train_test():
    TRAIN_PATH = 'input/saftey_efficay_myopiaTrain.csv'
    TEST_PATH = 'input/saftey_efficay_myopiaTest.csv'
    train = pd.read_csv(TRAIN_PATH, low_memory=False)
    test = pd.read_csv(TEST_PATH, low_memory=False)
    return train,test

def remove_columns(train, test, columns_to_remove):
    train = train.drop(columns=columns_to_remove)
    test = test.drop(columns=columns_to_remove)
    return train, test

def preprocess_data(train, test, pipeline):
    """
    pipleline - pipeline to fit_transform the concat dummies before spliting to train & test
    return: train & train dummies after pipeline fit_transform
    """
    n_train = train.shape[0]
    concat = pd.concat(objs=[train, test], axis=0, sort=False)
    concat_dummies = pd.get_dummies(concat)
    concat_dummies = pipeline.fit_transform(concat_dummies)
    train_dummies = copy(concat_dummies[:n_train])
    test_dummies = copy(concat_dummies[n_train:])
    return train_dummies, test_dummies


In [103]:
train, test = read_train_test()
train = train.dropna(axis=0, how='all') # remove missing rows
y_train = train.Class
train = train.drop(columns='Class')

columns_to_drop = ['Pre_L_Pupil_Day', 'T_L_Actual_AblDepth']
train, test = remove_columns(train, test, columns_to_drop)

print("train.shape without missing rows =", train.shape)
print("class count:\n", y_train.value_counts())

train.shape without missing rows = (30451, 49)
class count:
 0.0    29949
1.0      502
Name: Class, dtype: int64


In [104]:
##### add here new steps to preprocess
preprocessing = Pipeline([
                ('scaler', StandardScaler()),
                ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
                ])
X_train, X_test = preprocess_data(train, test, preprocessing)

In [105]:
### add here new steps before predict (TODO: undersample)
evaluation =  Pipeline([
                ('smote', SMOTE(random_state=42, ratio=0.8, n_jobs=-1)),
                ('rf', RandomForestClassifier(n_estimators=100, max_depth=8, n_jobs=-1))
                ])
evaluation.get_params().keys()

dict_keys(['memory', 'steps', 'smote', 'rf', 'smote__k_neighbors', 'smote__kind', 'smote__m_neighbors', 'smote__n_jobs', 'smote__out_step', 'smote__random_state', 'smote__ratio', 'smote__sampling_strategy', 'smote__svm_estimator', 'rf__bootstrap', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__min_impurity_decrease', 'rf__min_impurity_split', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_estimators', 'rf__n_jobs', 'rf__oob_score', 'rf__random_state', 'rf__verbose', 'rf__warm_start'])

In [106]:
#### Cross Validation

n_splits = 10 # number of folds (stratified KFold)
n_iter = 100 # number of parameter settings that are sampled

### add here new hyperparameters
rf_hyperparams = {
    'rf__n_estimators': np.arange(start=50, stop=500, step=10),
    'rf__max_features': ['auto', 'sqrt'],
    'rf__max_depth': list(np.arange(5,30,5)) + [None],
    'rf__min_samples_split': [2, 5, 10]
}
smote_hyperparams = {
    'smote__k_neighbors': np.arange(2,10)
    # 'smote__ratio': ??? TODO
}

rs = RandomizedSearchCV(evaluation, 
                        param_distributions={**rf_hyperparams, **smote_hyperparams},
                        scoring='roc_auc',
                        n_iter=n_iter, 
                        cv=StratifiedKFold(n_splits=n_splits),
                        n_jobs=-1)

rs.fit(X_train, y_train)  

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
          error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('smote', SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=-1,
   out_step='deprecated', random_state=42, ratio=0.8,
   sampling_strategy='auto', svm_estimator='deprecated')), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
          ..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'rf__n_estimators': array([ 50,  60,  70,  80,  90, 100, 110, 120, 130, 140, 150, 160, 170,
       180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300,
       310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430,
       440, 450, 460, 470, 480, 490]), 'rf__max_features': ['auto', 'sqrt'], 'rf__max_dep

In [107]:
print('best params:\n\t{}'.format(rs.best_params_))
print('best auc score:\n\t{:.4f}'.format(rs.best_score_))
pred = rs.predict_proba(X_test)[:,1]

ids = np.arange(1, pred.shape[0]+1)
rf_df = pd.DataFrame({"Id" : ids, "Class" : pred})
rf_df.to_csv("out/{}.csv".format("rf_with_tune"), index=False)


best params:
	{'smote__k_neighbors': 3, 'rf__n_estimators': 390, 'rf__min_samples_split': 2, 'rf__max_features': 'sqrt', 'rf__max_depth': 5}
best auc score:
	0.5961
